# Método dos Elementos Finitos - Trabalho 2

Este trabalho visa aplicar o MEF a uma estrutura de treliças.

Para tanto, deverá ler um arquivo em disco com os dados da treliça para resolver via código em Python.

Após ler o arquivo, o código abaixo gera um objeto de classe própria com as propriedades do sistema contido no arquivo.

A seguir o arquivo lido e suas principais características.

In [1]:
%display latex
import functions as fct

In [2]:
tr = fct.trelica("trelica.json")
print('Sistema de {} nós'.format(tr.n))
print('A interligação entre os nós pode ser representada pela seguinte matriz:\n{}\nOnde os elementos fora da diagonal principal representam os valores de 1/EA da barra entre os nós respectivos.'.format(tr.barras))
print('A matriz abaixo representa as cargas no nó:\n{}\nOnde a linha representa o nó e a coluna as dimensões x, y e z, respectivamente.'.format(tr.cargas))
print('Abaixo, temos as seguintes configurações para os nós (coordenada x,y,z e se há apoio na direção):')
for n in tr.nos:
    print(n.tag, '({}, {}, {})'.format(n.x, n.y, n.z), '--> [Rx:{}    Ry:{}    Rz:{}]'.format(n.Rx, n.Ry, n.Rz))

Sistema de 3 nós
A interligação entre os nós pode ser representada pela seguinte matriz:
[[0.    0.001 0.001]
 [0.001 0.    0.001]
 [0.001 0.001 0.   ]]
Onde os elementos fora da diagonal principal representam os valores de 1/EA da barra entre os nós respectivos.
A matriz abaixo representa as cargas no nó:
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 1. -1.  0.]]
Onde a linha representa o nó e a coluna as dimensões x, y e z, respectivamente.
Abaixo, temos as seguintes configurações para os nós (coordenada x,y,z e se há apoio na direção):
Nó 1 (0.0, 0.0, 0.0) --> [Rx:True    Ry:True    Rz:False]
Nó 2 (1.0, 0.0, 0.0) --> [Rx:False    Ry:True    Rz:False]
Nó 3 (1.0, 1.0, 0.0) --> [Rx:False    Ry:False    Rz:False]


In [3]:
x,N = var('x,N')
u = function('u')(x)
eq = diff(u,x,2) == N
eq.show()

diff(u(x), x, x) == N

In [5]:
sol = desolve(eq,u,[0,0,0],contrib_ode=True)
sol.show()

ValueError: Unable to determine independent variable, please specify.